In [2]:
import pandas as pd
import numpy as np
fama_french_data= pd.read_csv('C:\\Users\\zihan\\Desktop\\Files\\Research\\F-F_Research_Data_Factors.csv')
path_target = 'C:/Users/zihan/Desktop/Files/Research/riskfreerate.npz'
print('Saving data to ' + path_target)
np.savez(path_target, fama_french_data=fama_french_data)
print('Finished! ')

Saving data to C:/Users/zihan/Desktop/Files/Research/riskfreerate.npz
Finished! 


In [1]:
# Unconditional sort
import numpy as np
import matplotlib.pyplot as plt
loaddata = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
data = loaddata['processed_data']
date= loaddata['stockdate']
fama_french_data= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\riskfreerate.npz')
fama_french_data= fama_french_data['fama_french_data']

In [2]:
import pandas as pd
size_accrual_fama= pd.read_csv('C:\\Users\\zihan\\Desktop\\Files\\Research\\25_Portfolios_ME_AC_5x5.csv')
path_target = 'C:/Users/zihan/Desktop/Files/Research/size_accrual_fama.npz'
print('Saving data to ' + path_target)
np.savez(path_target, size_accrual_fama=size_accrual_fama)
print('Finished! ')

Saving data to C:/Users/zihan/Desktop/Files/Research/size_accrual_fama.npz
Finished! 


In [3]:
size_accrual_fama= np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\size_accrual_fama.npz')
size_accrual_fama= size_accrual_fama['size_accrual_fama']
print (fama_french_data.shape)
r_f= fama_french_data[:,4]
# normalize the interest rate because Fama French is 100%
r_f= r_f/100
date= fama_french_data[:,0]
date_start= np.asscalar(np.argwhere(date==198406))
date_end= np.asscalar(np.argwhere(date==201405))
portfolio_date= size_accrual_fama[:,0]
portfolio_date_start= np.asscalar(np.argwhere(portfolio_date==198406))
portfolio_date_end= np.asscalar(np.argwhere(portfolio_date==201405))
fama_portfolio= size_accrual_fama[portfolio_date_start:portfolio_date_end+1,1:26]/100
print (fama_portfolio.shape)
portfolio= fama_portfolio
portfolio= fama_portfolio-(r_f[date_start:date_end+1:,np.newaxis])

(1098, 5)
(360, 25)


In [53]:
M = 5
returns= data[:,:,1]
returns= returns- (r_f[date_start:date_end+1:,np.newaxis])
print (returns.shape)
accural= data[:,:,20]
size= data[:,:,17]
T,N= returns.shape
portfolio = np.zeros((T-1,25))
for i in range(T-1):
    return_t= returns[i+1,:]
    basis= np.zeros((N, M*M))
    for j in range(M):
        for k in range(M):
            quantile_accural_lower= np.percentile(accural[i,:], 100.0/M*(j))
            quantile_accural_upper= np.percentile(accural[i,:], 100.0/M*(j+1))
            quantile_size_lower= np.percentile(size[i,:], 100.0/M*(k))
            quantile_size_upper = np.percentile(size[i,:], 100.0/M*(k+1))
            basis[:,j*M+k]= (accural[i,:]>= quantile_accural_lower) & (accural[i,:]<= quantile_accural_upper)& (size[i,:]>= quantile_size_lower)& (size[i,:]<= quantile_size_upper)
    portfolio[i,:]= return_t.dot(basis)

(360, 304)


In [54]:
# Perform Risk Premium PCA, and calculate out of sample PCA
import scipy.linalg as la
K = 11
Factor = np.zeros((int(T / 2), K))
start = int(T / 2) - 1
gamma = 20
for t in range(int(T / 2)):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[:(t + start), :]
    # print (portfolio_t.shape)
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    # print (covmat_t.shape)
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_mean_t.shape)
    # print ((portfolio_mean_t.T.dot(portfolio_mean_t)).shape)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / (t + start - 1) * gamma
    # print (covmat_t.shape)
    variance_t, loading_t = la.eigh(covmat_t, eigvals=(0, K))
    Factor[t, :] = portfolio[t + start, :].dot(loading_t[:, :K])
    # print (Factor.shape)
output = []
for K in range(1, 11):
    mu = np.mean(Factor[:, :K], axis=0)
    covmat_factor = np.cov(Factor[:, :K].T)
    if K == 1:
        output.append(abs(mu) * covmat_factor ** (-1 / 2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))

Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
&0.06&0.07&0.08&0.08&0.11&0.13&0.17&0.18&0.18&0.18


In [55]:
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# # Dimension of portfolio: T*M
# covmat = portfolio_demean.T.dot(portfolio_demean) / (T - 2)
# variance, loading = np.linalg.eig(covmat)
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
print (portfolio_mean.shape)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
loading= loading*np.sqrt(N)
output= []
loading_chosen = loading[:, :11]
Factor = portfolio.dot(loading_chosen)/N
for i in range(1, 11):
    mu = np.mean(Factor[:,:i], axis=0)
    covmat_factor = np.cov(Factor[:,:i].T)
    if i==1: 
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

(1, 25)
&0.22&0.24&0.28&0.28&0.29&0.29&0.30&0.30&0.30&0.33


In [56]:
# Rolling window approach
import scipy.linalg as la
K = 11
window= 120
Factor = np.zeros((T-1-window, K))
gamma = 10
for t in range(T-1-window):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    # print (portfolio_t.shape)
    covmat_t = portfolio_t.T.dot(portfolio_t) 
    # print (covmat_t.shape)
    portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_mean_t.shape)
    # print ((portfolio_mean_t.T.dot(portfolio_mean_t)).shape)
    covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    # print (covmat_t.shape)
    variance_t, loading_t = la.eigh(covmat_t, eigvals=(0, K))
    Factor[t, :] = portfolio[t + window, :].dot(loading_t[:, :K])
    # print (Factor.shape)
output = []
for K in range(1, 11):
    mu = np.mean(Factor[:, :K], axis=0)
    # print (mu.shape)
    covmat_factor = np.cov(Factor[:, :K].T)
    # print (covmat_factor.shape)
    if K == 1:
        output.append(abs(mu) * covmat_factor ** (-1 / 2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu.T, np.linalg.inv(covmat_factor)), mu)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))


Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
&0.08&0.11&0.11&0.11&0.11&0.14&0.18&0.19&0.21&0.22


In [52]:
# Rolling window approach
import scipy.linalg as la
K = 11
window= 120
Factor = np.zeros((T-1-window, K))
gamma = 100
for t in range(T-1-window):
    if t % 20 == 0:
        print('Have gone through iteration %d' % (t))
    portfolio_t = portfolio[t:(t + window), :]
    # portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
    portfolio_demean_t = portfolio_t - np.mean(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_t.shape)
    # covmat_t = portfolio_t.T.dot(portfolio_t) 
    # print (covmat_t.shape)
    # portfolio_mean_t = np.sum(portfolio_t, axis=0, keepdims= True)
    # print (portfolio_mean_t.shape)
    # print ((portfolio_mean_t.T.dot(portfolio_mean_t)).shape)
    # covmat_t = covmat_t + portfolio_mean_t.T.dot(portfolio_mean_t) / window * gamma
    # print (covmat_t.shape)
    covmat_t= portfolio_demean_t.T.dot(portfolio_demean_t)
    variance_t, loading_t = la.eigh(covmat_t, eigvals=(0, K))
    Factor[t, :] = portfolio[t + window, :].dot(loading_t[:, :K])
    # print (Factor.shape)
output = []
for K in range(1, 11):
    mu = np.mean(Factor[:, :K], axis=0)
    # print (mu.shape)
    covmat_factor = np.cov(Factor[:, :K].T)
    # print (covmat_factor.shape)
    if K == 1:
        output.append(abs(mu) * covmat_factor ** (-1 / 2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu.T, np.linalg.inv(covmat_factor)), mu)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print(''.join(twodecimals))


Have gone through iteration 0
Have gone through iteration 20
Have gone through iteration 40
Have gone through iteration 60
Have gone through iteration 80
Have gone through iteration 100
Have gone through iteration 120
Have gone through iteration 140
Have gone through iteration 160
Have gone through iteration 180
Have gone through iteration 200
Have gone through iteration 220
&0.08&0.11&0.11&0.11&0.11&0.14&0.18&0.19&0.21&0.22
